In [1]:
import os
from pathlib import Path
import numpy as np
import pyvista as pv
# import project_heart as ph
from project_heart.modules.geometry import Geometry
pv.set_jupyter_backend("pythreejs")
from project_heart.lv import LV

In [2]:
dir = Path("C:/Users/igorp/University of South Florida/Wenbin Mao - Igor/LV_Meshes/Heart_models")
# dir = Path("C:/Users/igornobrega/USF_Research/University of South Florida/Wenbin Mao - Igor/LV_Meshes/Heart_models")
files = os.listdir(dir)

file_id = 3 # 5 has a problem

print("File:", files[file_id])

lv = LV()
lv.from_pyvista_read(dir/files[file_id], identifier="elemTag", threshold=[0, 1])

File: Full_Heart_Mesh_12.vtk


In [4]:
lv.identify_surfaces(
  
  ab_ql=0.03, 
  ab_qh=0.70,
  
  alpha_atr=0.20,  # coeff for radial distance computation
  alpha_mtr=0.7,
  
  beta_atr=0.15,  # coeff for second radial distance computation
  beta_mtr=0.30,
  
  gamma_atr=89,
  gamma2_mtr=25,
  
  phi_atr=79,
  epi_angle=100
)

plotter = pv.Plotter(lighting='three lights')
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
plotter.add_points(lv.get_virtual_node("MITRAL"), color="red", point_size=300)
plotter.add_points(lv.get_virtual_node("AORTIC"), color="purple", point_size=300)
plotter.add_mesh(lv.mesh, 
                  scalars="LV_SURFS", 
                  cmap="Set2", 
                  opacity=1.0, 
                  show_edges=False,
                  ambient=0.2, 
                  diffuse=0.5, 
                  specular=0.5, 
                  specular_power=90,
                  #  smooth_shading=True,
                  )
plotter.enable_shadows()
plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…